<a href="https://colab.research.google.com/github/Gregory-lab-eng/python_Vistula/blob/main/Profile_picture_validator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!pip install gradio_client

In [2]:
from transformers import BlipForQuestionAnswering, BlipProcessor


In [ ]:
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [ ]:
from PIL import Image
import gradio as gr

def save_and_process(img):

    image = Image.fromarray(img)
    image.save("uploaded_image.png")
    img_url = '/content/uploaded_image.png'

    question = "Is it a person?"
    inputs = processor(img_url, question, return_tensors="pt")
    outputs = model.generate(**inputs)
    model_answer = processor.decode(outputs[0], skip_special_tokens=True)

    if model_answer == "yes":
        reply = "This is a photo of a person you can proceed"
    else:
        reply = "Please upload a photo of a person"
    return reply


demo = gr.Interface(
    fn=save_and_process,
    inputs=gr.Image(type="numpy"),
    outputs="text"
)

demo.launch()


sprawdzanie modelu na różnych typach person


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_photo(img):
  plt.imshow(img)
  plt.axis('off')
  plt.show()

In [ ]:


img = mpimg.imread('/content/595f89000d1d3_o_large.jpg')
show_photo(img)
save_and_process(img)

In [ ]:
img = mpimg.imread('/content/download (1).jpg')
show_photo(img)
save_and_process(img)

In [ ]:
img = mpimg.imread('/content/download.jpg')
show_photo(img)
save_and_process(img)

In [ ]:
img = mpimg.imread('/content/istockphoto-1074169430-170667a.jpg')
show_photo(img)
save_and_process(img)

### Widać że model w 100% rozpoznaje różne typy